<a href="https://colab.research.google.com/github/GuptaTanay/Customer-Churn-Attrition/blob/main/Bank_Customer_Attrition_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from google.colab import files
# file  = files.upload()

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
#Reading the file
data = pd.read_csv('BankCustomers.csv')

In [4]:
# Visualize the columns
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Independant and Dependant feature split
X = data.iloc[:,3:-1]
Y = data.iloc[:,-1]

In [6]:
# Check which columns are Categorical
X.select_dtypes(include=['object'])

,Geography,Gender
0,France,Female
1,Spain,Female
2,France,Female
3,France,Female
4,Spain,Female
...,...,...
9995,France,Male
9996,France,Male
9997,France,Female
9998,Germany,Male


In [7]:
# Perform One Hot Encoding on Categorical features
states = pd.get_dummies(X.Geography, drop_first= True)
gender = pd.get_dummies(X.Gender, drop_first= True)

In [8]:
# Concat the X with OHE columns
X = pd.concat([X, states, gender], axis =1)
#axis = 1 represants concatenation at column level

In [9]:
#Visualize the data
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,0,1


In [10]:
# Drop the categorical features
X.drop(['Geography', 'Gender'], axis =1, inplace = True)

In [11]:
X

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,39,5,0.00,2,1,0,96270.64,0,0,1
9996,516,35,10,57369.61,1,1,1,101699.77,0,0,1
9997,709,36,7,0.00,1,0,1,42085.58,0,0,0
9998,772,42,3,75075.31,2,1,0,92888.52,1,0,1


In [12]:
# Perform train test split
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size= 0.2, random_state = 98)

In [13]:
# Perform feature scaling to Gaussian Normal distribution
# import sklearn
# dir(sklearn)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X)
test_X = scaler.transform(test_X)

In [14]:
# Visualise the scaled data set in the form of a dataset
visualise_df = pd.DataFrame(train_X, columns = X.columns)
visualise_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,1.118214,-0.660286,0.698533,-1.223645,0.807760,0.646092,0.971895,0.102031,-0.576196,-0.571769,-1.093339
1,-0.251047,-0.850372,-0.681882,0.838746,0.807760,0.646092,0.971895,-1.043227,1.735522,-0.571769,0.914630
2,-0.086324,-0.185072,0.353429,-1.223645,0.807760,-1.547768,-1.028918,0.737359,-0.576196,-0.571769,0.914630
3,-1.651194,2.761256,-1.372090,0.298499,-0.914084,-1.547768,0.971895,0.284904,1.735522,-0.571769,-1.093339
4,1.025557,1.430656,1.388741,1.280734,0.807760,-1.547768,0.971895,0.279833,1.735522,-0.571769,0.914630
...,...,...,...,...,...,...,...,...,...,...,...
7995,-0.879054,-0.850372,0.008326,-1.223645,-0.914084,0.646092,-1.028918,-0.786777,-0.576196,-0.571769,0.914630
7996,0.098990,0.195099,0.008326,-0.773870,-0.914084,0.646092,-1.028918,-0.531505,-0.576196,1.748958,-1.093339
7997,-0.106914,-0.755329,1.733845,1.274497,-0.914084,0.646092,-1.028918,-0.149733,-0.576196,-0.571769,0.914630
7998,0.088694,-0.470201,1.043637,0.481842,0.807760,-1.547768,0.971895,1.430537,-0.576196,-0.571769,0.914630


In [ ]:
# Change the shape for the LSTM
train_X = np.reshape(train_X, (train_X.shape[0], train_X.shape[1], 1))

## Let us create LSTM now!

In [15]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [16]:
# Initialise the LSTM 

classifier = Sequential()

In [17]:
classifier.add(LSTM(units = 50, return_sequences = True, input_shape = (train_X.shape[1],1)))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50, return_sequences = True))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50, return_sequences = True))
classifier.add(Dropout(0.2))

classifier.add(LSTM(units = 50))
classifier.add(Dropout(0.2))

classifier.add(Dense(units = 1))

classifier.compile(optimizer = 'adam', loss = 'mean_squared_error')

classifier.fit(train_X, train_y, epochs = 100, batch_size = 32)


Epoch 1/100
250/250 [==============================] - 27s 20ms/step - loss: 0.1682
Epoch 2/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1558
Epoch 3/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1512
Epoch 4/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1468
Epoch 5/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1460
Epoch 6/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1398
Epoch 7/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1386
Epoch 8/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1341
Epoch 9/100
250/250 [==============================] - 5s 19ms/step - loss: 0.1246
Epoch 10/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1226
Epoch 11/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1281
Epoch 12/100
250/250 [==============================] - 5s 20ms/step - loss: 0.1204


In [20]:
# Change the shape for LSTM
test_X = np.reshape(test_X, (test_X.shape[0], test_X.shape[1], 1))

In [21]:
pred_y = classifier.predict(test_X)

In [22]:
pred_y = (pred_y > 0.5)

## Let us check the accuracy now!

In [23]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [24]:
cm = confusion_matrix(test_y, pred_y)
cm

array([[1467,  114],
       [ 200,  219]])

In [25]:
accuracy = accuracy_score(test_y, pred_y)
accuracy

0.843